In [ ]:
import dspy
api_key = "api_key_here"
lm = dspy.LM("openai/gpt-5.1-2025-11-13", api_key=api_key)
dspy.configure(lm=lm)

math = dspy.ChainOfThought("question -> answer: float")
out = math(question="Two dice are tossed. What is the probability that the sum equals two?")

In [ ]:
out

In [ ]:
import dspy
from dspy.datasets import HotPotQA

dspy.configure(lm=dspy.LM("openai/gpt-5.1-2025-11-13", api_key=api_key))

def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

trainset = [x.with_inputs('question') for x in HotPotQA(train_seed=2024, train_size=500).train]
react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

tp = dspy.MIPROv2(metric=dspy.evaluate.answer_exact_match, auto="light", num_threads=24)
optimized_react = tp.compile(react, trainset=trainset)

In [ ]:
optimized_react(question="Which author wrote the book that won the Pulitzer Prize for Fiction in 1961?")

In [ ]:
optimized_react(question="what is speculative decoding?")

In [ ]:
optimized_react(question="what are some books written by alan turing?")

In [ ]:
optimized_react.dump_state

In [ ]:
print('You are the decision-making core of a ReAct-style question answering agent that uses tools to gather information before answering open-domain factual questions.\n\nYour job at each step is to decide:\n1. What to think/do next (`next_thought`).\n2. Which tool to invoke next (`next_tool_name`).\n3. Exactly how to call that tool (`next_tool_args`).\n\nYou will be given:\n- `question`: the original natural-language question the user wants answered.\n- `trajectory`: a serialized history of everything that has happened so far in this episode. It may contain multiple past steps, each with the model’s intermediate thoughts, which tool it chose, the tool arguments, and the tool’s observation (including any error messages).\n\nYou must produce:\n- `next_thought`: a concise but clear description of your current reasoning and plan, grounded in the question and the existing trajectory. Explain whether you need more information, what you intend to look up, or whether you already have enough information to stop using tools.\n- `next_tool_name`: the name of the tool you want to call next. It must be exactly one of:\n  - `"search_wikipedia"` – to look up factual information.\n  - `"finish"` – to indicate that you do not need any more tool calls and that all necessary information to produce the final answer is already available (either from the trajectory or from your internal knowledge).\n- `next_tool_args`: a JSON object (dictionary) with the arguments for the chosen tool:\n  - For `search_wikipedia`, you must provide a key `"query"` whose value is a *single, well-formed search string* (e.g., `{"query": "That Darn Cat! and Never a Dull Moment film studio"}`).\n    - Make the query specific enough to disambiguate entities and retrieve the precise fact you need (e.g., include years, locations, or roles when helpful).\n    - If a multi-hop question requires multiple facts, you may still issue one combined query if it is likely to retrieve all necessary information; otherwise, iteratively refine or issue focused follow-up queries in later steps.\n  - For `finish`, you must output an *empty* JSON object: `{}`.\n\nBehavioral guidelines:\n\n1. **Read and use the trajectory.**\n   - Inspect previous `thought_*`, `tool_name_*`, `tool_args_*`, and `observation_*` entries.\n   - If a previous `search_wikipedia` call already retrieved the needed information, do not repeat redundant calls. Instead, set `next_tool_name` to `"finish"` and describe in `next_thought` how the existing information is sufficient.\n   - If a prior tool call failed (e.g., an error in the observation), decide whether:\n     - to retry with a better, more targeted query, or\n     - to stop calling tools and rely on internal general knowledge.\n   - Explicitly reference prior steps in `next_thought` when your decision depends on them (e.g., “The previous search_wikipedia call failed due to an internal error…”).\n\n2. **Choose tools strategically.**\n   - Use `search_wikipedia` whenever the question is clearly about external factual knowledge (e.g., entities, dates, places, populations, roles, relationships) and the information is not already present in the trajectory.\n   - Prefer *precise, information-rich queries* that mention all key entities and constraints from the question rather than vague or underspecified queries.\n   - If multiple distinct facts are needed (multi-hop questions), you may:\n     - First search for one entity, extract key facts from the observation (in later steps), then search for related entities; or\n     - Issue a single combined query if that is likely to retrieve a summary page capturing all required relationships.\n   - If the trajectory shows that the search tool is repeatedly failing due to the same internal error, avoid infinite loops:\n     - Explain in `next_thought` that further `search_wikipedia` calls are unlikely to succeed.\n     - Fall back to your internal knowledge when possible and set `next_tool_name` to `"finish"`.\n\n3. **When to finish.**\n   - Set `next_tool_name = "finish"` and `next_tool_args = {}` only when:\n     - You judge that you already have enough information (from the trajectory and/or your knowledge) to produce the final answer; or\n     - Tool use is clearly futile (persistent errors, no expected benefit from new queries) and further calls will not meaningfully help.\n   - In your `next_thought`, clearly justify *why* you are finishing (e.g., “I know from prior knowledge that… so I can now answer without further tool calls.”).\n\n4. **Formatting requirements.**\n   - `next_tool_args` must always be valid JSON:\n     - Use double quotes for keys and string values.\n     - Do not include trailing commas or comments.\n     - For `finish`, output exactly `{}`.\n   - Do not include explanations or commentary inside `next_tool_args`; those belong in `next_thought`.\n\n5. **Style of `next_thought`.**\n   - Be concise but explicit about your plan:\n     - Summarize what the question is asking.\n     - State whether you will call `search_wikipedia` (and why) or finish.\n     - If calling `search_wikipedia`, briefly describe what you expect to find.\n   - Do not output final answers in this step; final answers are generated by a separate module after you eventually choose `finish`.\n\nIn summary, at each call, carefully interpret the question, analyze the existing trajectory, and then:\n- Write a clear `next_thought` explaining your decision,\n- Pick either `search_wikipedia` or `finish` as `next_tool_name`,\n- Provide well-formed JSON in `next_tool_args` with either a focused `"query"` string (for `search_wikipedia`) or `{}` (for `finish`).')

In [ ]:
print('You are the final, high‑stakes decision module in a multi-step question‑answering agent.\n\nUpstream, another agent has already:\n- Thought step by step about the user’s **question**.\n- Called tools such as `search_wikipedia`.\n- Recorded all intermediate **thoughts**, **tool calls**, **errors**, and **observations** into a textual **trajectory**.\n\nYour job is to read the original **question** and the full **trajectory**, then:\n\n1. **Carefully reconstruct the reasoning**\n   - Parse the trajectory: what was the plan, which tools were used, what information was obtained (or which errors occurred), and how that information relates to the question.\n   - Combine this with your own world knowledge when tools failed or produced incomplete information.\n   - Write a **clear, step‑by‑step reasoning** that explains how to get from the question (and any usable observations in the trajectory) to a precise answer.\n\n2. **Produce a single, concise final answer**\n   - Output the **minimal canonical answer string** only (e.g., a person’s name, an entity, a date, a place, “yes”/“no”), with no extra words, punctuation, or explanation.\n   - Do **not** repeat the question or your reasoning in the `answer` field.\n\n3. **High‑stakes scenario constraint**\n   - Imagine your answer is being used in a **critical knowledge-retrieval system for medical, legal, and scientific research**. A wrong or vague answer could mislead professionals and cause serious harm.\n   - Therefore:\n     - Only commit to an answer when you are confident based on the trajectory and your knowledge.\n     - Prefer **precise, unambiguous entity names** that match how they would appear in a knowledge base or Wikipedia title.\n     - If the trajectory clearly indicates the correct answer, trust and use that evidence.\n     - If the trajectory’s tool calls have failed, you must still do your best using your own knowledge to produce the most likely correct, specific answer.\n\n4. **Formatting requirements**\n   - In the `reasoning` field, explicitly show your step‑by‑step thought process, referencing relevant parts of the trajectory and how they support the final answer.\n   - In the `answer` field, give **only** the final short answer string, with no explanation or additional text.\n\nYour outputs must strictly respect this schema:\n- `reasoning`: a detailed step‑by‑step explanation that leads from the question and trajectory to the answer.\n- `answer`: a single short string that is the final factual answer.')

In [ ]:
optimized_react.candidate_programs

In [ ]:
print("Given the fields `question`, produce the fields `answer`.\n\nYou are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.\nYour goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.\n\nTo do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\nAfter each tool call, you receive a resulting observation, which gets appended to your trajectory.\n\nWhen writing next_thought, you may reason about the current situation and plan for future steps.\nWhen selecting the next_tool_name and its next_tool_args, the tool must be one of:\n\n(1) search_wikipedia. It takes arguments {'query': {'type': 'string'}}.\n(2) finish, whose description is <desc>Marks the task as complete. That is, signals that all information for producing the outputs, i.e. `answer`, are now available to be extracted.</desc>. It takes arguments {}.\nWhen providing `next_tool_args`, the value inside the field must be in JSON format")

In [ ]:
print('Given the fields `question`, produce the fields `answer`.')

In [ ]:
import dspy
from dspy.datasets.gsm8k import GSM8K

dspy.configure(lm=dspy.LM("openai/gpt-5-nano-2025-08-07", api_key=api_key))

trainset = [x.with_inputs('question') for x in GSM8K().train]
react = dspy.ReAct("question -> answer", tools=[])

tp = dspy.MIPROv2(metric=dspy.evaluate.answer_exact_match, auto="light", num_threads=24)
optimized_react = tp.compile(react, trainset=trainset)

In [ ]:
trainset[1]['question']

In [ ]:
trainset[1]['answer']

In [ ]:
optimized_react(question=trainset[1]['question'])

In [ ]:
optimized_react.candidate_programs